In [1]:
from functools import partial
import os
import numpy as np
import cirq
from cirq import(
    I,
    Circuit,
    NoiseModel,
    DensityMatrixSimulator,
    BitFlipChannel,
    PhaseFlipChannel,
    measure,
)

from mitiq import zne
from mitiq.benchmarks import generate_rb_circuits
from mitiq.zne.scaling import fold_global

In [10]:
def execute(circuit, shots, correct_bitstring):
    """Executes the input circuit(s) and returns ⟨A⟩, where
    A = |correct_bitstring⟩⟨correct_bitstring| for each circuit.
    """
    circuit_to_run = circuit.copy()

    circuit_to_run += measure(*sorted(circuit.all_qubits()), key="m")
    backend = DensityMatrixSimulator()

    result = backend.run(circuit_to_run, repetitions=shots)
    expval = result.measurements["m"].tolist().count(correct_bitstring) / shots
    return expval

Pauli errors are applied with probability of the logcial error rate, which is related to the code distance by $$L_{ERR}=0.03*(P_{ERR}/P_{TH})^{(D+1)/2}$$

where $P_{ERR}$ is the physical error rate and $P_{TH}$ is a threshold error rate.

In [3]:
def gen_noise_model(PERR, distance):
    """Create sweepable Pauli noise model."""
    PTH = 0.009
    LERR = 0.03 * (PERR / PTH) ** int((distance + 1) / 2)
    return LERR # model as single-qubit errors remaining after correction 


class PauliNoiseModel(NoiseModel):

    def __init__(self, error_rate):
        self.error_rate = error_rate

    def noisy_operation(self, op):
        error_rate = self.error_rate
        channel = BitFlipChannel(error_rate).on_each(op.qubits)
        channel += PhaseFlipChannel(error_rate).on_each(op.qubits)
        return [op, channel]

In [12]:
def merge_func(op1, op2):
    return True

In [13]:
def noisy_execute(circ, noise_level, shots, correct_bitstring):
    qubits = circ.all_qubits()
    copy = Circuit()
    for moment in circ.moments:
        idle = False
        for q in qubits:
            # every moment every qubit gets a single-qubit noise op
            if not moment.operates_on_single_qubit(q):
                idle = True
                op_to_circ = Circuit(PauliNoiseModel(noise_level).noisy_operation(cirq.I(q)))
                merged_op = cirq.merge_operations_to_circuit_op(op_to_circ, merge_func)
                copy.append(moment.with_operations(merged_op.all_operations()))
                break
        if not idle:
            copy.append(moment)
    noisy_circ = copy.with_noise(PauliNoiseModel(noise_level))
    return execute(noisy_circ, shots=shots, correct_bitstring=correct_bitstring)

In [14]:
def distance_scaled_execute(circ, distance, base_noise_level, shots, correct_bitstring):
    LERR = gen_noise_model(base_noise_level, distance)
    return noisy_execute(circ, LERR, shots, correct_bitstring)

In [16]:
def scale_shots(num_device_qubits, scaled_distance, base_shots, n_qubits_circuit):
    used_qubits = n_qubits_circuit * scaled_distance ** 2
    return base_shots * int(num_device_qubits / used_qubits)

In [2]:
num_trials = 100

base_shots = 10000
device_size = 1200
p_err=0.006
n_qubits = 2
correct_bitstring = [0] * n_qubits
depth = 20

scale_factors = [1, 3, 5, 7]
fac = zne.PolyFactory(scale_factors, order=3)

d_array = [21, 19, 17, 15, 13, 11, 9, 7, 5]

In [ ]:
circuits = generate_rb_circuits(n_qubits, depth, trials=num_trials)
trial_results = np.zeros((num_trials, 5, len(d_array) - 3)) # row: trial, column: scaling technique, page: distance (high to low)

for d_ind in range(len(d_array) - 3):
    print(f"On distance {d_array[d_ind]}") 
    for trial in range(num_trials):
        if trial in np.linspace(0, num_trials, 11):
            print(f"    On trial {trial}")  
        executor = partial(distance_scaled_execute, distance=d_array[d_ind], base_noise_level=p_err, shots=scale_shots(device_size, d_array[d_ind], base_shots, n_qubits), correct_bitstring=correct_bitstring)
        fac.run(circuits[trial], executor, scale_noise=fold_global)
        trial_results[trial, :-1, d_ind] = fac.get_expectation_values()
        trial_results[trial, -1, d_ind] = distance_scaled_execute(circuits[trial], d_array[d_ind], p_err, 4 * scale_shots(device_size, d_array[d_ind], base_shots, n_qubits), correct_bitstring)
    p_err_string = str(p_err).replace(".", "")[1:]
    np.savetxt(os.path.join(
                f"/Users/mistywahl/Documents/GitHub/zne-distance-scaling/notebooks/data_scale_shots_perr_{p_err_string}",
                f"depth{depth}_distance{d_array[d_ind]}.txt",
            ),
            trial_results[:, :, d_ind],
        )

In [ ]:
unscaled_results = np.zeros((num_trials, 2, 3))
result_ind = 0
for d_ind in range(len(d_array) - 3, len(d_array)):
    print(f"On distance {d_array[d_ind]}") 
    for trial in range(num_trials):
        if trial in np.linspace(0, num_trials, 11):
            print(f"   On trial {trial}") 
        unscaled_results[trial, 0, result_ind] = distance_scaled_execute(circuits[trial], d_array[d_ind], base_noise_level=p_err, shots=scale_shots(device_size, d_array[d_ind], base_shots, n_qubits), correct_bitstring=correct_bitstring)
        unscaled_results[trial, 1, result_ind] = distance_scaled_execute(circuits[trial], d_array[d_ind], base_noise_level=p_err, shots=4 * scale_shots(device_size, d_array[d_ind], base_shots, n_qubits), correct_bitstring=correct_bitstring)
    p_err_string = str(p_err).replace(".", "")[1:]
    np.savetxt(os.path.join(
                f"/Users/mistywahl/Documents/GitHub/zne-distance-scaling/notebooks/data_scale_shots_perr_{p_err_string}",
                f"depth{depth}_distance{d_array[d_ind]}.txt",
            ),
            unscaled_results[:, :, result_ind],
        )
    result_ind += 1

In [4]:
def calculate_folding_exp_vals(scale_factors, exp_vals, num_trials):
    """Perform extrapolation to zero noise limit on expectation values obtained
    with noise scaling by unitary folding. Return mean and standard deviation
    of the ZNE expectation values.
    """
    folding_values = np.zeros((num_trials, 1))

    for trial in range(num_trials):
        folding_values[trial] = fac.extrapolate(scale_factors, exp_vals[trial, : -1], order=3)

    return [np.mean(folding_values), np.std(folding_values)]

In [5]:
def distance_extrapolation(distance_scale_factors, ds_expectation_values):
    fac = zne.PolyFactory(scale_factors=distance_scale_factors, order=3)
    for s, v in zip(distance_scale_factors, ds_expectation_values):
        fac.push({"scale_factor": s}, v)
    result = fac.reduce()
    # _ = fac.plot_fit()
    return result

In [6]:
def calculate_ds_exp_vals(d_array, distance_indices, exp_vals, num_trials):
    """Perform extrapolation to zero noise limit on expectation values obtained
    with noise scaling by unitary folding. Return mean and standard deviation
    of the ZNE expectation values.
    """
    ds_values = np.zeros((num_trials, len(distance_indices)))
    # distance_scale_factors = [gen_noise_model(p_err, d_array[d]) / gen_noise_model(p_err, d_array[distance_indices[0][0]]) for d in distance_indices[0]]

    for count, d_ind in enumerate(distance_indices):
        distance_scale_factors = [gen_noise_model(p_err, d_array[di]) / gen_noise_model(p_err, d_array[d_ind[0]]) for di in d_ind]
        for trial in range(num_trials):
            ds_values[trial, count] = distance_extrapolation(distance_scale_factors, exp_vals[trial, d_ind])

    return [np.mean(ds_values, axis=0), np.std(ds_values, axis=0)]


In [12]:
num_trials = 100

base_shots = 10000
device_size = 1200
p_err=0.006
n_qubits = 2
correct_bitstring = [0] * n_qubits
depth = 30

scale_factors = [1, 3, 5, 7]
fac = zne.PolyFactory(scale_factors, order=3)

d_array = [21, 19, 17, 15, 13, 11, 9, 7, 5]

In [ ]:
circuits = generate_rb_circuits(n_qubits, depth, trials=num_trials)
trial_results = np.zeros((num_trials, 5, len(d_array) - 3)) # row: trial, column: scaling technique, page: distance (high to low)
for d_ind in range(len(d_array) - 3):
    print(f"On distance {d_array[d_ind]}") 
    for trial in range(num_trials):
        if trial in np.linspace(0, num_trials, 11):
            print(f"    On trial {trial}")  
        executor = partial(distance_scaled_execute, distance=d_array[d_ind], base_noise_level=p_err, shots=scale_shots(device_size, d_array[d_ind], base_shots, n_qubits), correct_bitstring=correct_bitstring)
        fac.run(circuits[trial], executor, scale_noise=fold_global)
        trial_results[trial, :-1, d_ind] = fac.get_expectation_values()
        trial_results[trial, -1, d_ind] = distance_scaled_execute(circuits[trial], d_array[d_ind], p_err, 4 * scale_shots(device_size, d_array[d_ind], base_shots, n_qubits), correct_bitstring)
    p_err_string = str(p_err).replace(".", "")[1:]
    np.savetxt(os.path.join(
                f"/Users/mistywahl/Documents/GitHub/zne-distance-scaling/notebooks/data_scale_shots_perr_{p_err_string}",
                f"depth{depth}_distance{d_array[d_ind]}.txt",
            ),
            trial_results[:, :, d_ind],
        )

In [ ]:
unscaled_results = np.zeros((num_trials, 2, 3))
result_ind = 0
for d_ind in range(len(d_array) - 3, len(d_array)):
    print(f"On distance {d_array[d_ind]}") 
    for trial in range(num_trials):
        if trial in np.linspace(0, num_trials, 11):
            print(f"   On trial {trial}") 
        unscaled_results[trial, 0, result_ind] = distance_scaled_execute(circuits[trial], d_array[d_ind], base_noise_level=p_err, shots=scale_shots(device_size, d_array[d_ind], base_shots, n_qubits), correct_bitstring=correct_bitstring)
        unscaled_results[trial, 1, result_ind] = distance_scaled_execute(circuits[trial], d_array[d_ind], base_noise_level=p_err, shots=4 * scale_shots(device_size, d_array[d_ind], base_shots, n_qubits), correct_bitstring=correct_bitstring)
    p_err_string = str(p_err).replace(".", "")[1:]
    np.savetxt(os.path.join(
                f"/Users/mistywahl/Documents/GitHub/zne-distance-scaling/notebooks/data_scale_shots_perr_{p_err_string}",
                f"depth{depth}_distance{d_array[d_ind]}.txt",
            ),
            unscaled_results[:, :, result_ind],
        )
    result_ind += 1